In [1]:
import math
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
import numpy as np
import pylab as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import time

c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is depr

In [12]:
# scale data
def scale(X, X_min, X_max):
    return (X - X_min)/(X_max-X_min)

NUM_FEATURES = 21
NUM_CLASSES = 3

learning_rate = 0.01
epochs = 2000
beta = [0, 1e-3, 1e-6, 1e-9, 1e-12]
batch_size = 32
num_neurons = 10
prob = 0.5
seed = 10
np.random.seed(seed)

#read train data

train_input = np.genfromtxt('ctg_data_cleaned.csv', delimiter= ',')
trainX, train_Y = train_input[1:, :21], train_input[1:,-1].astype(int)
#split dataset in 70:30
X_train, X_test, Ytrain, Ytest = train_test_split(trainX, train_Y, test_size=0.3, random_state=20, shuffle=True)

X_train = scale(X_train, np.min(X_train, axis=0), np.max(X_train, axis=0))
X_test = scale(X_test, np.min(X_test, axis=0), np.max(X_test, axis=0))

Y_train = np.zeros((Ytrain.shape[0], NUM_CLASSES))
Y_train[np.arange(Ytrain.shape[0]), Ytrain-1] = 1 #one hot matrix

Y_test = np.zeros((Ytest.shape[0], NUM_CLASSES))
Y_test[np.arange(Ytest.shape[0]), Ytest-1] = 1 #one hot matrix

n = X_train.shape[0]
    
# Create the model
x = tf.placeholder(tf.float32, [None, NUM_FEATURES])
y_ = tf.placeholder(tf.float32, [None, NUM_CLASSES])


In [14]:
kf = KFold(n_splits=5, random_state=20, shuffle=True)
N = len(X_train)

acc_test = []
acc_train = []

keep_prob = tf.placeholder(tf.float32)

# Build the graph for the deep net
	
#Build 1st layer to hidden layer
weights_to_h1 = tf.Variable(tf.truncated_normal([NUM_FEATURES, num_neurons], stddev=1.0/math.sqrt(float(NUM_FEATURES))), name='weights_to_h1')
biases_to_h1  = tf.Variable(tf.zeros([num_neurons]), name='biases_to_h1')
hidden1 = tf.nn.relu(tf.matmul(x, weights_to_h1) + biases_to_h1)
h1_dropout = tf.nn.dropout(hidden1, keep_prob)

#Build 2nd layer to hidden layer
weights_to_h2 = tf.Variable(tf.truncated_normal([num_neurons, num_neurons], stddev=1.0/math.sqrt(float(num_neurons))), name='weights_to_h2')
biases_to_h2  = tf.Variable(tf.zeros([num_neurons]), name='biases_to_h2')
hidden2 = tf.nn.relu(tf.matmul(h1_dropout, weights_to_h2) + biases_to_h2)
h2_dropout = tf.nn.dropout(hidden2, keep_prob)

#Build 3rd layer to output layer (softmax layer)
weights_to_o = tf.Variable(tf.truncated_normal([num_neurons, NUM_CLASSES], stddev=1.0/math.sqrt(float(num_neurons))), name='weights_to_o')
biases_to_o  = tf.Variable(tf.zeros([NUM_CLASSES]), name='biases_to_o')
logits  = tf.matmul(h2_dropout, weights_to_o) + biases_to_o

#L2 regularization with weight decay
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=logits)
regularization = tf.nn.l2_loss(weights_to_h2) + tf.nn.l2_loss(weights_to_o)
loss = tf.reduce_mean(cross_entropy + beta*regularization)

# Create the gradient descent optimizer with the given learning rate.
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train_op = optimizer.minimize(loss)

correct_prediction = tf.cast(tf.equal(tf.argmax(logits, 1), tf.argmax(y_, 1)), tf.float32)
accuracy = tf.reduce_mean(correct_prediction)
    
fold = 1
for train_index, test_index in kf.split(X_train):

    print("Fold number: " + str(fold))
    X_traink, X_testk = X_train[train_index], X_train[test_index]
    Y_traink, Y_testk = Y_train[train_index], Y_train[test_index]
        
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        train_acc = []
        test_acc = []

        for i in range(epochs):
                
            for start, end in zip(range(0, N, batch_size), range(batch_size, N, batch_size)):
                train_op.run(feed_dict={x: X_traink[start:end], y_: Y_traink[start:end], keep_prob: prob})

            train_acc.append(accuracy.eval(feed_dict={x: X_traink, y_: Y_traink, keep_prob: 1.0}))
            test_acc.append(accuracy.eval(feed_dict={x: X_testk, y_: Y_testk, keep_prob: 1.0}))

            if i % 100 == 0:
                print('iter %d: Train accuracy %g'%(i, train_acc[i]))
                print('iter %d: Test accuracy %g'%(i, test_acc[i]))
            
    #store accuracy of each fold
    acc_test.append(test_acc)
    acc_train.append(train_acc)
    fold += 1


Fold number: 1


InvalidArgumentError: Incompatible shapes: [32] vs. [5]
	 [[node gradients_7/add_44_grad/BroadcastGradientArgs (defined at <ipython-input-14-0ad73620cd57>:35) ]]

Original stack trace for 'gradients_7/add_44_grad/BroadcastGradientArgs':
  File "c:\python37\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\python37\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\python37\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\python37\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "c:\python37\lib\site-packages\ipykernel\kernelapp.py", line 563, in start
    self.io_loop.start()
  File "c:\python37\lib\site-packages\tornado\platform\asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "c:\python37\lib\asyncio\base_events.py", line 534, in run_forever
    self._run_once()
  File "c:\python37\lib\async